In [4]:
import pyro
import torch
import numpy as np
from pyro.infer.importance import Importance

# Whatsapp puzzle

Probability of a day being Sunday is givn by Bernoulli distribution
$$P(A) = p_{A}(x) = \mathrm{Bern}\left(x;1/7\right)$$

If the day is Sunday, number of messages $B$ per hour is given by the Poisson distribution
$$P(B|A=1) = p_{B}(x|A=1) = \mathrm{Poi}(x;3)$$
otherwise
$$P(B|A=0) = p_{B}(x|A=0) = \mathrm{Poi}(x;10)$$

If receives $4$ messages on a day, what is the probability that the day is Sunday
$$P(A=1|B=4)$$

# Calculate

In [5]:
def scale():
    weight = pyro.sample("weight", pyro.distributions.Normal(2.0, 1.0), is_cont=False)
    if weight < 2.2:
        m = pyro.sample("measurement_1", pyro.distributions.Normal(weight - 1, 0.75), is_cont=False)
    else:
        tmp = pyro.sample("measurement_2", pyro.distributions.Normal(weight + 1, 0.75), is_cont=False)
        m = pyro.sample("measurement_1", pyro.distributions.Normal(tmp + 1, 0.25), is_cont=False)
    pyro.sample("obs", pyro.distributions.Normal(1.7, 0.8), obs=m)
    return weight.item()



In [6]:
importance = Importance(scale, guide=None, num_samples=20000)
importance.run()

In [8]:
log_weights = np.array([w.item() for w in importance.log_weights])
values = np.array([t.nodes['weight']['value'].item() for t in importance.exec_traces])
weights = np.exp(log_weights)
np.sum(weights * values) / np.sum(weights)

1.6921990320583897

# Sample from posterior via systematic resampling

In [9]:
import math
def systematic_resampling(log_weights, values):
    import torch

    mx = max(log_weights)
    weight_sum = sum(math.exp(log_weight - mx) for log_weight in log_weights)
    u_n = torch.distributions.Uniform(0, 1).sample().item()
    sum_acc = 0.0
    resamples = []
    for (log_weight, value) in zip(log_weights, values):
        weight = math.exp(log_weight - mx) * len(values) / weight_sum
        sum_acc += weight
        while u_n < sum_acc:
            u_n += 1
            resamples.append(value)
    return resamples


In [11]:
resamples = systematic_resampling(log_weights, values)                   
print(np.mean(resamples))                                            

1.6919209121346475


In [1]:
from pyro.infer.importance import Importance
import pyro.infer.mcmc as pyromcmc
from pathlib import Path
import pyro
import torch
import numpy as np

def scale():
    weight = pyro.sample("weight", pyro.distributions.Normal(2.0, 1.0), is_cont=False)
    if weight < 2.2:
        m = pyro.sample("measurement_1", pyro.distributions.Normal(weight - 1, 0.75), is_cont=False)
    else:
        tmp = pyro.sample("measurement_2", pyro.distributions.Normal(weight + 1, 0.75), is_cont=False)
        m = pyro.sample("measurement_1", pyro.distributions.Normal(tmp + 1, 0.25), is_cont=False)
    pyro.sample("obs", pyro.distributions.Normal(1.7, 0.8), obs=m)
    return weight.item()

kernel = pyromcmc.NPDHMC(
    scale,
    step_size=0.1,
    num_steps=50,
    adapt_step_size=False,
)
count = 1000
mcmc = pyromcmc.MCMC(kernel, num_samples=count, warmup_steps=count // 10)
mcmc.run()
samples = mcmc.get_samples()
print(samples["measurement_1"].mean(0))
for key, value in samples.items():
    print(key, value.shape, value)
mcmc.summary() 

Sample: 100%|██████████| 110/110 [00:21,  5.01it/s, step size=1.00e-01, acc. prob=0.993]


tensor(-0.1879, grad_fn=<MeanBackward1>)
measurement_1 torch.Size([100]) tensor([-1.3919,  1.0081, -0.9919,  0.4081, -0.7919,  0.6081, -1.1919,  0.6081,
        -1.1919,  0.6081, -0.1919,  0.0081, -0.3919,  0.0081, -0.5919,  0.0081,
         0.0081, -0.5919,  0.0081, -0.5919,  0.2081, -0.5919,  0.4081, -0.5919,
         0.4081, -0.7919,  0.4081, -0.9919,  0.4081, -0.9919,  1.0081, -0.9919,
         1.0081, -1.5919,  1.4081, -1.1919,  0.6081, -1.1919,  0.6081, -1.1919,
         1.0081, -1.1919,  0.6081, -1.1919,  0.6081, -1.1919,  0.6081, -1.1919,
         0.8081, -1.1919,  0.6081, -0.7919,  0.8081, -1.3919,  0.8081, -1.3919,
         1.0081, -0.7919,  0.4081, -0.7919,  0.2081,  0.0081, -0.5919,  0.0081,
        -0.3919,  0.0081, -0.3919, -0.1919, -0.3919, -0.1919, -0.3919,  0.0081,
        -0.5919,  0.0081, -0.3919, -0.1919, -0.3919, -0.1919, -0.1919, -0.3919,
         0.4081, -0.9919,  2.0081,  0.0081, -0.5919,  0.0081, -0.3919, -0.1919,
        -0.1919,  0.2081, -0.7919,  0.4081, -0.

In [2]:
print(samples["weight"].mean(0))

tensor(1.2544, grad_fn=<MeanBackward1>)
